# Fallacy Search Analysis

In [14]:
%load_ext autoreload
%autoreload 2

from src.mafalda import get_mafalda_df, evaluate_responses, get_mean_metrics
import seaborn as sns

sns.set_theme()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Experiment 3.1: Fallacy Search

In [2]:
filename_e31 = 'data/mafalda_e31.csv'
df_mafalda_e31 = get_mafalda_df(filename_e31)
df_mafalda_e31.head()

[2024-11-15 12:50:35] Loaded existing mafalda dataframe from data/mafalda_e31.csv.


,text,labels,comments,sentences_with_labels,gpt_4o_mini_response
0,TITLE: Endless Ledge Skip Campaign for Alts PO...,"[[155, 588, slippery slope]]","['Slippery slope: P1 = poster, A = why not jus...",{'TITLE: Endless Ledge Skip Campaign for Alts ...,fallacies=[FallacyEntry(fallacy=<Fallacy.SLIPP...
1,"Two of my best friends are really introverted,...","[[84, 145, hasty generalization]]","[""Based on two people only, you can't draw gen...",{'Two of my best friends are really introverte...,fallacies=[FallacyEntry(fallacy=<Fallacy.HASTY...
2,TITLE: There is a difference between a'smurf' ...,"[[118, 265, false analogy]]","['False Analogy: X: Having an alt , Y: smurfin...",{'TITLE: There is a difference between a'smurf...,fallacies=[FallacyEntry(fallacy=<Fallacy.EQUIV...
3,TITLE: Discussion Thread (Part 3): 2020 Presid...,"[[107, 261, guilt by association], [107, 338, ...",['Circular reasoning: X = The status quo in Am...,{'TITLE: Discussion Thread (Part 3): 2020 Pres...,fallacies=[FallacyEntry(fallacy=<Fallacy.HASTY...
4,"America is the best place to live, because it'...","[[0, 78, circular reasoning]]",['Circular reasoning: X=America is the best pl...,"{'America is the best place to live, because i...",fallacies=[FallacyEntry(fallacy=<Fallacy.CIRCU...


### Scoring and Sanity Check

In [11]:
evaluate_responses(df_mafalda_e31)

Failed to match span for fallacy Fallacy.HASTY_GENERALIZATION:
span: If you sell big amounts at 6k, he tried to bring the price even lower and cause a total sell off.
text: TITLE: Now We Know Who Sold the Bottom at $6k And Tried To Crash Bitcoin - Spoiler alert: it was Mt.Gox trustee POST: can anyone explain the part where it says he tried to crash the market at 6k?.... i dont get the reasoning behind this? POST: If you sell at high prices, that makes sense, doesn't it? If you sell at bottom, you're either panicking or trying to crash the price of the asset you're selling. So if he sold big amounts at 6k, he tried to bring the price even lower and cause a total sell off. That is my logic.

Failed to match span for fallacy Fallacy.HASTY_GENERALIZATION:
span: Well, democracy gave us Trump to the whole world. I'll let others judge what democracy is. Lets see what other system gave us. Communism gave us the Kim dynasty that destroyed their own country. Fascism gave us Franco that kept his 

In [23]:
metrics = get_mean_metrics(df_mafalda_e31)
metrics.round(3)

,precision,recall,f1
gpt_4o_mini,0.294,0.609,0.282


In [12]:
test = 'test'

KeyboardInterrupt: 